# Introduction to functional data cleaning using nilearn

### Movement is the enemy of neuroimagers

In task-based fMRI, if participants move during task-relevant moments
- Get a huge **false task-related signal** that’s actually due to motion!

In resting-state fMRI, movement can induce **false correlations** between brain regions


Solving for this involves *modelling* our fMRI signal to be comprised of **true brain signal** and **confounder signals**. 


Our goal is to remove a majority (hopefully) of the **confounder signals** and acquire something *closer* to the **true signal**. 

This is achieved via **confound regression**, which is essentially fitting a linear model using confounds as regressors then subtracting it out from the signal


*****

In [1]:
import os
from nilearn import image as img
from nilearn import plotting as plot
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
import bids
%matplotlib inline

/home/jerry/.pyenv/versions/3.6.0/envs/scwg_neuroimaging/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


## Step 1: Implementing Confound Regression using FMRIPREP outputs
FMRIPREP estimates confounds for the functional image and outputs it into:

**sub-xxxxx_task-xxxx_space-xxxx_..._confounds.tsv**

Let's look in the functional directory for a confounds file

In [8]:
sub = '10171'
fmriprep_dir = '../data/ds000030/derivatives/fmriprep/'
layout = bids.BIDSLayout(fmriprep_dir,validate=False)

In [9]:
func_files = layout.get(subject=sub, datatype='func', task='rest', suffix='preproc')
mask_files = layout.get(subject=sub, datatype='func', task='rest', suffix='brainmask')
confound_files = layout.get(subject=sub, datatype='func', task='rest', suffix='confounds')

In [10]:
func_file = func_files[0].path
mask_file = mask_files[0].path
confound_file = confound_files[0].path

The confounds file is organized like an excel spread-sheet with multiple columns, each for a specific confound.

We can view these using pandas 

In [ ]:
import pandas as pd

In [ ]:
confound_df = pd.read_csv(confound_file,delimiter='\t')

In [ ]:
#Pandas load and view

Each column in this DataFrame <code>confound_df</code> represents a specific confound variable that is either estimated directly from head motion during the functional scan or other noise characteristics that may capture noise (non grey-matter signal for example). Each row represents values from a TR/sample. So the number of rows in your <code>confound_df</code> should match the number of TRs you have in the functional MR data. The choice of which confounds to use in functional imaging analysis is a source of large debate. We recommend that you check out these sources for a start:

1. https://www.sciencedirect.com/science/article/pii/S1053811917302288#f0005
2. https://www.sciencedirect.com/science/article/pii/S1053811917302288

For now we're going to replicate the pre-processing (mostly) from the seminal Yeo1000 17-networks paper:

https://www.ncbi.nlm.nih.gov/pubmed/21653723

### The (mostly, slightly modified) Yeo 2011 Pre-processing schema

#### Confound regressors
1. 6 motion parameters (X, Y, Z, RotX, RotY, RotZ) 
2. Global signal (GlobalSignal)
3. 2 Largest Principal components of non-grey matter (aCompCor01, aCompCor02)   

This is a total of 9 base confound regressor variables. Finally we add temporal derivatives of each of these signals as well (1 temporal derivative for each), the result is 18 confound regressors.

***
**Temporal Derivatives** are the changes in values across 2 consecutive samples. It represents change in signal over time. For example, when dealing with the confound variable "X", which represents motion along the "X" direction, the temporal derivative represents *velocity in the X direction*. 

***

#### Low/High pass filtering
1. Low pass filtering cutoff: 0.08 
2. High pass filtering cutoff: 0.009

Low pass filters out high frequency signals from our data. fMRI signals are slow evolving processes, any high frequency signals are likely due to noise 
High pass filters out any very low frequency signals (below 0.009Hz), which may be due to intrinsic scanner instabilities

#### Drop dummy TRs
During the initial stages of a functional scan there is a strong signal decay artifact, thus the first 4ish or so 
TRs are very high intensity signals that don't reflect the rest of the scan. Therefore we drop these timepoints. 

#### Censoring + Interpolation (leaving out)
Censoring involves removal and interpolation of high-movement frames from the fMRI data. Interpolation is typically done using sophisticated algorithms much like [Power et al. 2014](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3849338/). 

** We won't be using censoring + interpolation since its fairly complicated and would take up too much time **

***

### Setting up Confound variables for regression

#### Computing temporal derivatives for confound variables

In [ ]:
#Select confounds and view

In [ ]:
#Compute temporal derivative

In [ ]:
#View

As you might have noticed, we have NaN's in our {confound}_dt. This happens because there is no prior value to the first index to take a difference with, but this isn't a problem since we're going to be dropping 4 timepoints from our data and confounders anyway!

#### Dummy TR Drop


In [ ]:
#First we'll load in our data and check the shape

Recall that the fourth dimension represents frames/TRs(timepoints). We want to drop the first four timepoints entirely, to do so we use nibabel's slicer feature. We'll also drop the first 4 confound variable timepoints to match the functional scan

In [ ]:
#TR drop

In [ ]:
#Confound TR drop

### Applying confound regression

Now we'd like to clean our data of our selected confound variables. There are two ways to go about this: 

1. If you have nilearn version 0.5.0 or higher use <code>nilearn.image.clean_img(image,confounds,...)</code>
2. If you want full control over specific parts of the image you're cleaning use <code>nilearn.signal.clean(signals,confounds,...)</code> 

The first method is probably most practical and can be done in one line given what we've already set-up. However, in cases of very large datasets (HCP-style), the second method might be preferable for optimizing memory usage. 

We'll go over both
***

First note that both methods take an argument <code>confounds</code>. This is a matrix:

$$
\left.\left( 
\vphantom{ \begin{array}{c} 1 \\ 1 \\1 \\1 \\1 \end{array} }
\smash{ \underbrace{
                    \begin{array}{cccccc} 
                    a_1 & b_1 & c_1 & \cdots & x_1 & \\
                    a_2 & b_2 & c_2 & \cdots & x_2 &\\
                    a_3 & b_3 & c_3 & \cdots & x_3 &\\
                    \vdots & \vdots & \vdots & \dots & \vdots &\\
                    a_T & b_T & c_T & \cdots & x_T & 
                    \end{array}
                   }_{ \text{ # of confound variables }}
      }
\right)
\right\}\,T\text{ number of frames}
$$
<br></br>

Therefore our goal is to take our confound matrix and work it into a matrix of the form above. The end goal is a matrix with 147 rows, and columns matching the number of confound variables (9x2=18)

Luckily this is a one-liner!

In [ ]:
#Form a confound matrix


Let's clean our image!

#### Method 1: Using <code>nilearn.image.clean_img</code> 

First we'll describe a couple of this function's important arguments. Any argument enclosed in [arg] is optional

<code>nilearn.image.clean_img(image,confounds,[low_pass],[high_pass],[t_r],[mask_img],[detrend],[standardize])</code>

**Required**:
- <code>image</code>: The functional image (<code> func_img </code>)
- <code>confounds</code>: The confound matrix (<code> confounds </code>) 

**Optional**:
- <code>low_pass</code>: A low pass filter cut-off
- <code>high_pass</code> A high pass filter cut-off
- <code>t_r</code>: This is required if using low/high pass, the repetition time of acquisition (imaging metadata) 
- <code>mask_img</code> Apply a mask when performing confound regression, will speed up regression
- <code>detrend</code>: Remove drift from the data (useful for removing scanner instability artifacts) [default=True]
- <code>standardize</code>: Set mean to 0, and variance to 1 --> sets up data for statistical analysis [default=True]
*** 
**What we're using**: 

The Repetition Time of our data is 2 seconds, in addition since we're replicating (mostly) Yeo 2011's analysis: 
- high_pass = 0.009
- low_pass = 0.08
- detrend = True
- standardize = True

In addition we'll use a mask of our MNI transformed functional image (<code> mask </code>) to speed up cleaning 



In [ ]:
#Set some constants
high_pass= 0.009
low_pass = 0.08
t_r = 2

In [ ]:
#Clean!

In [ ]:
#Let's visualize our result on slice #51

***

#### Method 2: Using <code>nilearn.signal.clean</code>

The arguments to this function are almost identical to <code>nilearn.image.clean_img</code>: 

<code>nilearn.signal.clean(signals,confounds,[low_pass],[high_pass],[t_r],[detrend],[standardize]</code> 

The only difference being:

- <code>signals</code>: The resting state signals matrix
- no <code>mask_img</code> argument exists, we'll have to pick which voxels to apply confound regression to ourselves!

In [ ]:
#Load in nilearn.signal
from nilearn import signal as sgl

#### Step 1: Set up our data signals matrix
Recall our data is a 4D array, with the fourth dimension represented as time and the other 3 dimensions representing the (x,y,z) coordinate of a particular voxel. We want to convert this to a matrix represented as the following:

$$
\left.\left( 
\vphantom{ \begin{array}{c} 1 \\ 1 \\1 \\1 \\1 \end{array} }
\smash{ \underbrace{
                    \begin{array}{cccccc} 
                    a_1 & b_1 & c_1 & \cdots & x_1 & \\
                    a_2 & b_2 & c_2 & \cdots & x_2 &\\
                    a_3 & b_3 & c_3 & \cdots & x_3 &\\
                    \vdots & \vdots & \vdots &\cdots & \vdots &\\
                    a_T & b_T & c_T & \cdots & x_T & 
                    \end{array}
                   }_{x*y*z \text{ voxels }}
      }
\right)
\right\}\,T\text{ number of frames}
$$

<br><br>
- The **number of columns represents the total number of voxels (x\*y\*z)**, each column being a single voxel
- The **number of rows represents the number of timepoints**

So we need to *reshape* our data to match this:

In [ ]:
#First we pull out our data as a numpy arrays


#Then we get x,y,z dimensions


#Then we get total number of voxels across all frames, x*y*z


#Then we reshape to the correct size, note that matrix is flipped on its side
#Where the number of rows matches the number of voxels instead of time-series


In [ ]:
#Now we flip it over (switch columns and rows) 


#### Step 2: Masking our signals matrix (Optional)

Using this <code>signals</code> matrix will work, but we'll also be cleaning *background/non-brain* voxels which may slow down our cleaning if we're cleaning hundreds (or *thousands* in the case of HCP) of images. 

To speed up the process we should only apply cleaning to the *subset of voxels that belong to the brain*. We can do this by masking out which voxels to apply cleaning to. This is equivalent to using <code>mask_img</code> in **Method 1** except we'll be doing this manually - it'll be slightly more complicated!.

To apply the mask to our <code>signals</code>, we want our mask to be in a similar format to our <code>signals</code>

In [ ]:
#Load in mask image


#Pull out data matrix


#Get dimensions of mask image


#Reshape the data so that each column corresponds to a voxel


The end result is a 1-dimensional array where each element corresponds to a voxel. Any element that is equal to 0 corresponds to a background voxel and any element corresponding to a brain voxel is equal to 1. 
To select which voxels to clean we'll find all the indices where <code>flattened_mask</code> equals 1

In [ ]:
#Get the voxel indices (corresponding to column #s in our signals) that are non-zero (brain voxels) 


#### Step 3: Cleaning our data
First we'll set up our filtering variables

In [ ]:
#Set up variables for confound regression
low_pass = 0.08 
high_pass = 0.009
rep_time= 2

Applying the actual cleaning once our data is set up will be very similar to <code>nilearn.image.clean_img</code> in form. The major difference will be that to mask our data we'll pick which indices to apply <code>nilearn.signal.clean</code> to using <code>brain_voxels</code>. 

*** 

In practice we'll do the following: 
1. Create an matrix of zeros matching our <code>signals</code>, we'll call it <code>cleaned_signals</code>
2. In the voxels (columns) corresponding to <code>brain_voxels</code> write in the cleaned time-series

Specifically, step 2 will be accomplished using the following:

<code>cleaned_signals[:,brain_voxels] = nilearn.signal.clean(signals[:,brain_voxels],...)</code> 

Notice, <code>signals[:,brain_voxels]</code>, this does two things: 
1. Select all rows - **rows correspond to frames and we want all frames**
2. Select the columns using <code>brain_voxels</code>. Remember **columns represent voxels, and <code>brain_voxels</code> corresponds to brain voxels**. 

Then <code>cleaned_signals[:,brain_voxels]</code> will select which voxels to write our cleaned time-series into. Notice that doing this sets our background voxels to 0. 

In [ ]:
#First create a matrix of zeroes that matches our signals matrix


#Apply only to brain voxels


In [ ]:
#Now we can reconstruct our volume, just perform operations in reverse
#Step 1: Flip it back


#Step 2: Reshape it back into a 4D time-series


In [ ]:
#The cleaned image generated from nilearn.signal.clean


Let's compare this to <code>cleaned_img</code> which is what we generated using <code>nilearn.image.clean_img</code>. They should be identical

### Done!

Hopefully by now you've learned what confound regression is, and how to perform it in nilearn using 2 different methods. We'd like to note that there are many more methods to perform confound regression (simultaneous signal extraction + confound regression for example) but all those methods fundamentally rely on what you've done here. 

In addition, performing confound regression on *functional volumes*, is also not the only way to do data cleaning. More modern methods involve applying confound regression on *functional surfaces*, however, those methods are too advanced for an introductory course to functional data analysis and involve tools outside of python. 

If you're interested in surface-based analysis we recommend that you check out the following sources:

1. https://edickie.github.io/ciftify/#/
2. https://www.humanconnectome.org/software/connectome-workbench
3. [The minimal preprocessing pipelines for the Human Connectome Project](https://www.ncbi.nlm.nih.gov/pubmed/23668970)